# Lektion 6 - Introduktion till deep learning-ramverk
## Assignment: Tensors and a tiny neural network

Instructions:
1. Choose PyTorch or Keras
2. Keep the model small and run quickly
3. Explain what each block does
"""



## Task 1: Tensor operations
TODO: Create a tensor with shape (2, 3)

In [7]:
import torch 


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

A = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

print(A)
print("A shape:", A.shape)

tensor([[1., 2., 3.],
        [4., 5., 6.]])
A shape: torch.Size([2, 3])


TODO: Reshape to (3, 2)


In [9]:
B = A.reshape(3,2)
print(B)
print("\n B shape:", B.shape)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

 B shape: torch.Size([3, 2])


TODO: Demonstrate broadcasting with another tensor


In [11]:
C = torch.tensor([10.0, 20.0]) #shape (2,)
D = B + C
print("\nC shape:", C.shape)
print("D = B + C (broadcasted):")
print(D)


C shape: torch.Size([2])
D = B + C (broadcasted):
tensor([[11., 22.],
        [13., 24.],
        [15., 26.]])



## Task 2: Build a tiny neural network
TODO: Use MNIST or iris (flattened)

In [2]:
import torch 
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# 1) Data: Load + split + scale
iris = load_iris()
X = iris.data  #shape(150, 4)
y = iris.target #shape; (150,)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features (important for stable traning)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

TODO: Build a model with:
- 1 hidden layer (ReLU)
- output layer (softmax or sigmoid)

In [3]:
class TinyNet(nn.Module):
    def __init__(self, in_dim=4, hiddemn_dim=16, out_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hiddemn_dim), # "dense" layer
            nn.ReLU(),  # Activation
            nn.Linear(hiddemn_dim, out_dim) # output logits (no softmax here) 
        )
    def forward(self, x):
        return self.net(x)

model = TinyNet()

# Loss + optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ---------------------------
# 3) Train loop: Few epochs
# ---------------------------

def accuracy(logits, y_true):
    preds = torch.argmax(logits, dim=1)
    return (preds == y_true).float().mean().item()

epochs = 30
for epoch in range(1, epochs + 1):
    model.train()

    # Forward
    logits = model(X_train_t)
    loss = criterion(logits, y_train_t)

    # Backwards
    optimizer.zero_grad()
    loss.backward()

    # Eval (on testset)
    model.eval()
    with torch.no_grad():
        test_logits = model(X_test_t)
        test_acc = accuracy(test_logits, y_test_t)
    
    if epoch in {1, 5, 10, 20, 30}:
        train_acc = accuracy(logits.detach(), y_train_t)
        print(f"{epoch:02d} | loss={loss.item():4f} | train_acc={train_acc:.3f} | test_acc{test_acc:.3f}")

# Final Eval
model.eval()
with torch.no_grad():
    final_test_acc = accuracy(model(X_test_t), y_test_t)
print("\nFinal test accuracy:", round(final_test_acc, 3))

01 | loss=1.078162 | train_acc=0.375 | test_acc0.400
05 | loss=1.078162 | train_acc=0.375 | test_acc0.400
10 | loss=1.078162 | train_acc=0.375 | test_acc0.400
20 | loss=1.078162 | train_acc=0.375 | test_acc0.400
30 | loss=1.078162 | train_acc=0.375 | test_acc0.400

Final test accuracy: 0.4



## Task 3: Compare APIs
TODO: Write 4-6 comment lines on:
- where PyTorch feels lower level
- where Keras feels higher level



- PyTorch feels lower level because you write the training loop yourself and explicitly control
forward-pass, backward-pass and optimizer steps, which makes the data flow visible.

- It is easy to inspect tensors, shapes and gradients during execution, which
suits debugging and research where you often violate “standard flows”.

- Keras feels higher level because much is abstracted away behind model.compile
and model.fit, where training logic, metrics and callbacks are handled automatically.

- This makes it fast to build and train models with few lines of code, but at the expense of
detailed control when you want to do something unusual or experimental.